Dans ce script, nous allons tester différents modèles de type convolutionnelsur le dataset 

résultat:
* l'utilisation des batchs rend l'inférence globale plus rapide
* un modèle a besoin de beaucoup de temps / parametres pour arriver à un certain niveau d'accuracy

In [1]:
import retinoto_py as fovea
args = fovea.Params()
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	 user laurentperrinet	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 1998 has been set.


Params(image_size=224, do_mask=False, num_epochs=5, n_train_stop=0, seed=1998, batch_size=64, label_smoothing=0.0, model_name='resnet18', do_scratch=False, shuffle=True, num_workers=2, verbose=True)

In [ ]:
VAL_DATA_DIR = args.DATAROOT / 'Imagenet_full' / 'val'
val_loader, class_to_idx, idx_to_class = fovea.get_loader(args, VAL_DATA_DIR)
model = fovea.load_model(args)
model.eval()
print(f"Model {args.model_name} loaded.")

# testing different networks

In [ ]:
args = fovea.Params()
from retinoto_py import all_models

json_filename = args.data_cache / '13_model_comparison.json'

if json_filename.exists():
    results = fovea.pd.read_json(json_filename)
else:
    all_results = []
    for model_name in fovea.all_models:
        tic = fovea.time.time()
        args = fovea.Params(model_name=model_name)
        model = fovea.load_model(args)
        param_stats = fovea.count_parameters(model)
        total_layers = fovea.count_layers(model)
        accuracy = fovea.get_validation_accuracy(args, model, val_loader)
        toc = fovea.time.time()
        this_result = {'model_name': model_name,
                                'accuracy': accuracy,
                                'wall_clock_time': toc-tic,
                                'total_parameters': param_stats['total_parameters'],
                                'trainable_parameters': param_stats['trainable_parameters'],
                                'total_layers': total_layers}
        all_results.append(this_result)
    results = fovea.pd.DataFrame(all_results)
    results.to_json(json_filename, orient='index', indent=2)


In [16]:
json_filename.rename?


Signature: json_filename.rename(target)
Docstring:
Rename this path to the target path.

The target path may be absolute or relative. Relative paths are
interpreted relative to the current working directory, *not* the
directory of the Path object.

Returns the new Path instance pointing to the target path.
File:      /opt/homebrew/Cellar/python@3.13/3.13.9_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pathlib/_local.py
Type:      method